# 基于lsa的文本相似度计算

#数据的导入与清理

In [1]:
##加载相关包
import jieba
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.metrics import silhouette_score ,calinski_harabasz_score,davies_bouldin_score

In [2]:
plot_kwds = {'alpha' : 0.7, 's': 40, 'linewidths':0,'cmap':'Set3'}

In [3]:
#导入数据
data=pd.read_excel('F:/data/tdata/社研.xls')
data=list(data['Summary-摘要'])
data

['基于拉图尔对"社会"的本体论批判以及赫兹菲尔德的"社会诗学"理论,本文对一个川南集镇的茶馆生活进行了民族志考察。不同于既有的"公共空间"范式,本文认为中国茶馆并非一个表征自治、理性和反抗精神的政治空间;相反,茶馆作为人们相互交往的"表演空间"、打牌消遣的"游戏空间"、谈论是非长短的"话语空间",其首要特征在于诗性(poetics)而非政治(politics)。茶馆不仅蕴藏了丰富的表演技术、游戏形式和话语策略;同时,作为联结自我与社会、结构与阈限、话语和实践的诗性空间,中国茶馆所表征的公共性与西方理论中的公共领域亦有着重要区别。',
 '宿舍是城市化、工业化进程中的重要制度创造。本文通过对20世纪30、40年代恒源纱厂女工宿舍的研究发现,国家、工厂和女工为应对各自的危局与困境,基于各自的结构势态分别对宿舍进行文化范畴的重新定义,结果却促成了工人宿舍共同体的产生。本研究探求女工宿舍共同体的生产,目的是发现工人以正式制度为基础营造有利于自身生活的非正式制度的基本逻辑,进而理解城市化、工业化进程中的制度再造现象,为以人为本的城市建设工作提供新的视野。',
 '本文利用一项结合社区线上和线下数据的混合研究设计,探索了兼具媒体、社会网络和政务平台等技术特征的社区新媒体与社区治理之间的互构关系。研究表明,居民日常的新媒体资源网络提升了社区治理水平,而官方正式平台却收效不佳。进一步分析指出,层级化的社区治理结构使官方平台的内容生产与居民社区性信息和互动需求分离,从而导致技术功能的"收缩"。因此,社区建设中信息技术和治理结构的协调发展至关重要。',
 '随着全球化的深入,日益频繁的城市间跨国政策转移正深刻地改变着全球城市的治理模式,将地方治理机制连接至全球城市政策共同体。城市在其中贡献并吸收各类治理方案,成为全球治理体系中最活跃的参与者与推动者。然而,传统的政策转移研究框架具有国家中心主义特征,因而未能为这些现象提供一个比较城市的分析视角。因此,有学者提出了聚焦于城市的政策流动概念,强调从跨学科视角关注城市政策跨国流动过程中的全球关系及空间—社会建构,从而推动了政策转移研究的流动性转向。本文从背景、理论基础以及现实关照三个方面对政策流动框架进行了梳理,并试图提出这一框架下未来的研究议题,以期推动国内比较城市政策的研究,促进政策研究的跨学科对话。',
 '改革开放后出生的中国新生代

文本数据清理，主要包括分词、去除标点符号停用词等

In [4]:
# 加载停用词
def get_stop_words(stop_words_dir):
    stop_words = []

    with open(stop_words_dir, 'r', encoding='utf-8') as f_reader:
        for line in f_reader:
            line = delete_r_n(line)
            stop_words.append(line)

    stop_words = set(stop_words)
    return stop_words

In [5]:
# 去掉空字符
def delete_r_n(line):
    return line.replace('\r', '').replace('\n', '').strip()

In [6]:
# 加载停用词,stopwords为停用词列表
stopwords = get_stop_words(r'F:/data/tdata/stopwords.txt')

In [7]:
text = [' '.join(re.findall(r'[\u4e00-\u9fa5]+', doc.encode('utf-8').decode('utf-8')))
          for doc in data]
text

['基于拉图尔对 社会 的本体论批判以及赫兹菲尔德的 社会诗学 理论 本文对一个川南集镇的茶馆生活进行了民族志考察 不同于既有的 公共空间 范式 本文认为中国茶馆并非一个表征自治 理性和反抗精神的政治空间 相反 茶馆作为人们相互交往的 表演空间 打牌消遣的 游戏空间 谈论是非长短的 话语空间 其首要特征在于诗性 而非政治 茶馆不仅蕴藏了丰富的表演技术 游戏形式和话语策略 同时 作为联结自我与社会 结构与阈限 话语和实践的诗性空间 中国茶馆所表征的公共性与西方理论中的公共领域亦有着重要区别',
 '宿舍是城市化 工业化进程中的重要制度创造 本文通过对 世纪 年代恒源纱厂女工宿舍的研究发现 国家 工厂和女工为应对各自的危局与困境 基于各自的结构势态分别对宿舍进行文化范畴的重新定义 结果却促成了工人宿舍共同体的产生 本研究探求女工宿舍共同体的生产 目的是发现工人以正式制度为基础营造有利于自身生活的非正式制度的基本逻辑 进而理解城市化 工业化进程中的制度再造现象 为以人为本的城市建设工作提供新的视野',
 '本文利用一项结合社区线上和线下数据的混合研究设计 探索了兼具媒体 社会网络和政务平台等技术特征的社区新媒体与社区治理之间的互构关系 研究表明 居民日常的新媒体资源网络提升了社区治理水平 而官方正式平台却收效不佳 进一步分析指出 层级化的社区治理结构使官方平台的内容生产与居民社区性信息和互动需求分离 从而导致技术功能的 收缩 因此 社区建设中信息技术和治理结构的协调发展至关重要',
 '随着全球化的深入 日益频繁的城市间跨国政策转移正深刻地改变着全球城市的治理模式 将地方治理机制连接至全球城市政策共同体 城市在其中贡献并吸收各类治理方案 成为全球治理体系中最活跃的参与者与推动者 然而 传统的政策转移研究框架具有国家中心主义特征 因而未能为这些现象提供一个比较城市的分析视角 因此 有学者提出了聚焦于城市的政策流动概念 强调从跨学科视角关注城市政策跨国流动过程中的全球关系及空间 社会建构 从而推动了政策转移研究的流动性转向 本文从背景 理论基础以及现实关照三个方面对政策流动框架进行了梳理 并试图提出这一框架下未来的研究议题 以期推动国内比较城市政策的研究 促进政策研究的跨学科对话',
 '改革开放后出生的中国新生代 是深受重大社会历史变迁影响又在其中发挥重要作用的 社会代 高

In [8]:
text = [' '.join(jieba.lcut(doc))
          for doc in text]
text

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\liuhe\AppData\Local\Temp\jieba.cache
Loading model cost 0.631 seconds.
Prefix dict has been built succesfully.


['基于 拉图尔 对   社会   的 本体论 批判 以及 赫兹 菲尔德 的   社会 诗学   理论   本文 对 一个 川南 集镇 的 茶馆 生活 进行 了 民族志 考察   不同于 既有 的   公共 空间   范式   本文 认为 中国 茶馆 并非 一个 表征 自治   理性 和 反抗 精神 的 政治 空间   相反   茶馆 作为 人们 相互 交往 的   表演 空间   打牌 消遣 的   游戏 空间   谈论 是非 长短 的   话语 空间   其 首要 特征 在于 诗性   而 非 政治   茶馆 不仅 蕴藏 了 丰富 的 表演 技术   游戏 形式 和 话语 策略   同时   作为 联结 自我 与 社会   结构 与 阈限   话语 和 实践 的 诗性 空间   中国 茶馆 所 表征 的 公共性 与 西方 理论 中 的 公共 领域 亦 有着 重要 区别',
 '宿舍 是 城市化   工业化 进程 中 的 重要 制度 创造   本文 通过 对   世纪   年代 恒源 纱厂 女工 宿舍 的 研究 发现   国家   工厂 和 女工 为 应对 各自 的 危局 与 困境   基于 各自 的 结构 势态 分别 对 宿舍 进行 文化 范畴 的 重新 定义   结果 却 促成 了 工人 宿舍 共同体 的 产生   本 研究 探求 女工 宿舍 共同体 的 生产   目的 是 发现 工人 以 正式 制度 为 基础 营造 有利于 自身 生活 的 非正式 制度 的 基本 逻辑   进而 理解 城市化   工业化 进程 中 的 制度 再造 现象   为 以人为本 的 城市 建设 工作 提供 新 的 视野',
 '本文 利用 一项 结合 社区 线上 和 线下 数据 的 混合 研究 设计   探索 了 兼具 媒体   社会 网络 和 政务 平台 等 技术 特征 的 社区 新 媒体 与 社区 治理 之间 的 互构 关系   研究 表明   居民 日常 的 新 媒体 资源 网络 提升 了 社区 治理 水平   而 官方 正式 平台 却 收效 不佳   进一步 分析 指出   层级 化 的 社区 治理 结构 使 官方 平台 的 内容 生产 与 居民 社区性 信息 和 互动 需求 分离   从而 导致 技术 功能 的   收缩   因此   社区 建设 中 信息技术 和 治理 结

CountVectorizer
首先我们要明白，计算机是不能从文本字符串中发现规律的。只有将字符串编码为计算机可以理解的数字，计算机才有可能发现文本中的规律。

最容易理解的实现思路：

对文本编码，就是让词语与数字对应起来，建立基于给定文本的词典。（fit方法 ）
再根据词典对所有的文本数据进行转码。（transform方法）
scikit库的CountVectorize类就是这种思路。

In [9]:
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
wordcounter = CountVectorizer()

In [10]:
#学习特征（构建词典）fit  并转化为特征矩阵。
matrix=wordcounter.fit_transform(text)
print(matrix.toarray())
#查看下特征与词语对应关系
print(wordcounter.get_feature_names())

[[2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['一个', '一个多', '一个多元', '一中', '一书', '一二五', '一些', '一代', '一位', '一体', '一体化', '一例', '一元', '一元化', '一元论', '一刀切', '一切', '一周年', '一场', '一套', '一孩', '一定', '一家', '一对', '一席之地', '一张', '一弱', '一批', '一整套', '一文', '一方', '一方面', '一时', '一时期', '一是', '一本', '一束', '一条', '一样', '一次', '一步', '一流', '一炉', '一点', '一环', '一直', '一神', '一种', '一端', '一类', '一系列', '一级', '一组', '一股', '一胎', '一致', '一致性', '一般', '一般化', '一般性', '一般说来', '一词', '一起', '一部', '一部分', '一门', '一阶', '一面', '一项', '七八', '七大', '七年级', '万人', '万字', '万左右', '万种', '万能', '丈夫', '三个', '三元', '三农问题', '三十余年', '三十年', '三大', '三支', '三方', '三是', '三权分立', '三次', '三点', '三种', '三类', '三级', '三线', '三线建设', '三组', '三者', '三角', '三起', '三道', '三重', '上上下下', '上下', '上下级', '上乘', '上分', '上千年', '上升', '上升幅度', '上半叶', '上受', '上同', '上均', '上层', '上层建筑', '上市公司', '上应', '上有政策', '上海', '上海市', '上级', '上级领导', '上网', '上行', '上访', '上述', '上马', '下一代', '下一阶段', '下乡', '下列', '下向', '下层', '下岗工人', '下放', '下有对策', '下来', '下

In [11]:
#文档词频矩阵
pd.DataFrame(matrix.toarray(), columns = wordcounter.get_feature_names())

,一个,一个多,一个多元,一中,一书,一二五,一些,一代,一位,一体,...,黑匣子,黑箱,黑龙江,默认,默顿,黯然,鼓励,鼠疫,齐泽克式,齐美尔
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#奇异值分解

In [12]:
from sklearn.decomposition import TruncatedSVD  # LSA

In [13]:
lsa = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=200)
t1 = lsa.fit_transform(matrix)  # 训练并进行转化
print("--------lsa奇异值---------")
print(lsa.singular_values_)
print("--------500个文本，在10个话题向量空间下的表示---------")
print(t1)  # 500个文本，在10个话题向量空间下的表示

--------lsa奇异值---------
[103.2622642   53.16180215  48.29934524  47.02135774  38.68741134
  37.29236614  35.8242011   34.77262056  34.29919902  33.77560424]
--------500个文本，在10个话题向量空间下的表示---------
[[ 4.287069    1.39074035  0.31303695 ... -0.07248117  1.22116304
   0.85438312]
 [ 2.61141184 -1.60842677  0.03481214 ...  0.76264701  1.54178402
   2.02708533]
 [ 3.22696328 -0.79821422  0.98774845 ... -0.62376193  0.38422123
   0.0795373 ]
 ...
 [ 3.42355119 -1.58160642 -2.22777872 ...  1.5275033  -1.22488842
  -0.57326763]
 [ 1.96151019 -0.88355715  0.36289925 ... -0.39168561  1.29595473
   0.98916861]
 [ 3.57477425  1.32146602  0.05283063 ... -1.02557938  0.36946022
   0.20889381]]


In [14]:
###基于余弦相似度的文本相似性计算
from sklearn.metrics.pairwise import cosine_similarity

sim=cosine_similarity(t1)

In [15]:
sim

array([[1.        , 0.53810992, 0.56367406, ..., 0.22685748, 0.71568393,
        0.80040298],
       [0.53810992, 1.        , 0.66842338, ..., 0.19266253, 0.61604087,
        0.48421038],
       [0.56367406, 0.66842338, 1.        , ..., 0.23757335, 0.40439798,
        0.72169434],
       ...,
       [0.22685748, 0.19266253, 0.23757335, ..., 1.        , 0.1461989 ,
        0.23519575],
       [0.71568393, 0.61604087, 0.40439798, ..., 0.1461989 , 1.        ,
        0.35162863],
       [0.80040298, 0.48421038, 0.72169434, ..., 0.23519575, 0.35162863,
        1.        ]])

In [ ]:
##进一步探索与分析